In [1]:
from pdf_parse import Reader
from samco import Samco
import pprint, json
import pandas as pd
import numpy as np
import fitz

dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
#dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"

In [ ]:
""" SAMCO PDF FILE MAIN CODE"""

_reader = Reader(dir_path)

file_path = _reader.get_file_path(r'\files\samco\58_30-Jun-24_FS.pdf')
dry_path  = _reader.get_file_path(r'\output\DryRun.pdf')
fin_path  = _reader.get_file_path(r'\files\financial_indices.xlsx')
json_path = _reader.get_file_path(r'\output\dump.json')


_tata = Samco()
financial_indices = _reader.get_financial_indices(fin_path)
fund_data = _tata.fund_data

path,fund_names,imp_pages = _reader.check_and_highlight(file_path, financial_indices,fund_data)
_reader.save_pdf_data(imp_pages, fund_names)


user_input = input("Enter the desired pages: ")
pages = list(map(int,user_input.split(" ")))
bbox = _tata.content_bbox

data = _reader.get_clipped_data(file_path,pages,bbox, fund_names)
data = _reader.extract_span_data(data,[])
clean_data = _reader.process_text_data(data, _tata.data_cond)

nested_data, matrix = _reader.create_nested_dict(clean_data, 20.0, 10.0)

#GENERATE/EXTRACT

extracted_text = dict() #hv to write a function for this !!
for fund, items in nested_data.items():
    print(fund)
    _reader.generate_pdf_from_data(items, dry_path)
    extract_data = _reader.extract_data_from_pdf(dry_path)
    extracted_text[fund] = extract_data
    
#CREATE FINAL CONTENT    
final_text = dict()
for fund, items in extracted_text.items():
    
    content_dict = dict()
    for header, content in items.items():
        header_content = _tata.match_regex_to_content(header, content)
        content_dict.update(header_content)

    final_text[fund] = content_dict


#pprint.pprint(final_text)  
with open(json_path, 'w',encoding='utf-8') as file:
    json.dump(final_text,file,ensure_ascii=False, indent=4)
    print("\nJSON CREATED")

# print("\n___________ Code Successfully Run _______________")

In [ ]:
""" TATA PDF FILE MAIN CODE"""
from tata import Tata
_reader = Reader(dir_path)

file_path = _reader.get_file_path(r'\files\TataFactSheet2024.pdf')
dry_path  = _reader.get_file_path(r'\output\DryRun.pdf')
fin_path  = _reader.get_file_path(r'\files\financial_indices.xlsx')
json_path = _reader.get_file_path(r'\output\dump_tata.json')


_tata = Tata()
financial_indices = _reader.get_financial_indices(fin_path)
fund_data = _tata.fund_data

path,fund_names,imp_pages = _reader.check_and_highlight(file_path, financial_indices,fund_data)
_reader.save_pdf_data(imp_pages, fund_names)


user_input = input("Enter the desired pages: ")
pages = [ i for i in range(16,60)]+ [61,63] + [i for i in range(65,82)]
bbox = _tata.content_bbox

data = _reader.get_clipped_data(file_path,pages,bbox, fund_names)
data = _reader.extract_span_data(data,[])
clean_data = _reader.process_text_data(data, _tata.data_conditions)

nested_data, matrix = _reader.create_nested_dict(clean_data, 20.0, 10.0)

#GENERATE/EXTRACT

extracted_text = dict() #hv to write a function for this !!
for fund, items in nested_data.items():
    print(fund)
    _reader.generate_pdf_from_data(items, dry_path)
    extract_data = _reader.extract_data_from_pdf(dry_path)
    extracted_text[fund] = extract_data
    
# #CREATE FINAL CONTENT    
# final_text = dict()
# for fund, items in extracted_text.items():
    
#     content_dict = dict()
#     for header, content in items.items():
#         header_content = _tata.match_regex_to_content(header, content)
#         content_dict.update(header_content)

#     final_text[fund] = content_dict


#pprint.pprint(final_text)  
with open(json_path, 'w',encoding='utf-8') as file:
    json.dump(extracted_text,file,ensure_ascii=False, indent=4)
    print("\nJSON CREATED")


In [79]:
"""ASSET ONE 360 FILE MAIN CODE"""

from asset360 import One360


_reader = Reader(dir_path)
file_path = _reader.get_file_path(r'\files\one360.pdf')
dry_path  = _reader.get_file_path(r'\output\DryRun.pdf')
fin_path  = _reader.get_file_path(r'\files\financial_indices.xlsx')
json_path = _reader.get_file_path(r'\output\dump360.json')

_one360 = One360()

financial_indices = _reader.get_financial_indices(fin_path)
fund_data = _one360.fund_data
bbox = [[0,50,160,900]]

path,fund_names,imp_pages = _reader.check_and_highlight(file_path, financial_indices,fund_data)
_reader.save_pdf_data(imp_pages, fund_names)

#user_input = input("Enter the desired pages: ")
pages = [5,6,7,8,9,10,11]

data = _reader.get_clipped_data(file_path,pages,bbox, fund_names)
extract_data = _reader.extract_span_data(data,[])
data_conditions = [[8.0],-10791002,20.0]
cleaned_data = _reader.process_text_data(extract_data, data_conditions)
nested_data, matrix = _reader.create_nested_dict(cleaned_data, 20.0, 10.0)


extracted_text = dict() #hv to write a function for this !!
for fund, items in nested_data.items():
    print(fund)
    _reader.generate_pdf_from_data(items, dry_path)
    extract_data = _reader.extract_data_from_pdf(dry_path)
    extracted_text[fund] = extract_data


Doc saved at C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\pdf_report.xlsx
360 one focused equity fund
 pdf generated to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\DryRun.pdf
360 one flexicap fund
 pdf generated to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\DryRun.pdf
360 one quant fund
 pdf generated to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\DryRun.pdf
360 one elss tax saver nifty 50 index fund
 pdf generated to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\DryRun.pdf
360 one balanced hybrid fund
 pdf generated to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\DryRun.pdf
360 one dynamic bond fund
 pdf generated to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Informati

In [ ]:
""" BAJAJ FINSERV FILE MAIN CODE"""

In [72]:
file_path = _reader.get_file_path(r'\files\bajaj finserv.pdf')
doc = fitz.open(file_path)

count = doc.page_count
all_blocks = list()

for pgn in range(count):
    page = doc[pgn]
    
    blocks = page.get_text('dict')['blocks']
    images = page.get_images()
    filtered_blocks = [block for block in blocks if block['type']==0]
    sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
    all_blocks.append({
        "pgn":pgn,
        "blocks":sorted_blocks,
        "images": images
    })
    
    #draw lines
    
    lines = fitz.Rect()
    
doc.close()

In [73]:
all_blocks[17]['images']

[(465, 464, 824, 359, 8, 'Indexed', '', 'X140', 'FlateDecode')]

In [ ]:
for page_number, page in enumerate(doc, start=0):
    print(f"\n_____________{page_number}______________")
    images = page.get_images(full=True)  # Get all images on the page
    for img_index, img in enumerate(images, start=1):
        print(img)

In [ ]:
all_blocks[17]['blocks']